In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-08-25 04:32:10--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  80.0MB/s    in 0.3s    

2022-08-25 04:32:11 (80.0 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
dataframe_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

dataframe_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'str', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
dataframe_books.dropna(inplace=True)
dataframe_ratings.dropna(inplace=True)

In [ ]:
dataframe_ratings.isnull().sum()

user      0
isbn      0
rating    0
dtype: int64

In [ ]:
dataframe_books.isnull().sum()

isbn      0
title     0
author    0
dtype: int64

In [ ]:
firstValue = dataframe_ratings['user'].value_counts()
secondValue = dataframe_ratings['isbn'].value_counts()

dataframe = dataframe_ratings[~dataframe_ratings['user'].isin(firstValue[firstValue < 200].index)]
dataframe = dataframe[~dataframe['isbn'].isin(secondValue[secondValue < 100].index)]
dataframe.shape

(49781, 3)

In [ ]:
dataframe = dataframe.pivot_table(index=['user'],columns=['isbn'],values='rating').fillna(0).T

In [ ]:
dataframe.index = dataframe.join(dataframe_books.set_index('isbn'))['title']

In [ ]:
model = NearestNeighbors(metric='cosine')
model.fit(dataframe.values)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  title = dataframe.loc[book]
  distance, indice = model.kneighbors([title.values], n_neighbors=6)
  recommended_books = pd.DataFrame({
      'title' : dataframe.iloc[indice[0]].index.values,
      'distance' : distance[0]
  }).sort_values(by='distance', ascending=False).head(6).values
  return [book, recommended_books]

In [ ]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)

['The Queen of the Damned (Vampire Chronicles (Paperback))', array([['Catch 22', 0.793983519077301],
       ['The Witching Hour (Lives of the Mayfair Witches)',
        0.7448656558990479],
       ['Interview with the Vampire', 0.7345069050788879],
       ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
        0.5376338958740234],
       ['The Vampire Lestat (Vampire Chronicles, Book II)',
        0.517841100692749],
       ['The Queen of the Damned (Vampire Chronicles (Paperback))', 0.0]],
      dtype=object)]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677074670791626],
       ['The Lovely Bones: A Novel', 0.7234864234924316],
       ["Where the Heart Is (Oprah's Book Club (Paperback))", 0.0]],
      dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
